# Determinação de atributos das quadras com densidade construtiva a partir de dados do IPTU, série total

In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
import numpy as np

## Carregando e pré-processando dados do IPTU

In [2]:
# Padrões de nomenclatura de usos

usos = {
    "apartamento em condomínio":"apartamento",
    "cortiço (habitação coletiva subnormal)":"cortiço",
    "escritório/consultório em condomínio (unidade autônoma)":"escritório ou consultório",
    "loja em edifício em condomínio (unidade autônoma)":"loja em edifício em condomínio",
    "residência coletiva, exclusive cortiço (mais de uma residência no lote)":"residência coletiva (mais de uma residência no lote), exclusive cortiço",
    "posto de serviço (combustíveis)":"posto de serviço",
    "estacionamento e garagem, não em condomínio":"garagem (exclusive em prédio em condomínio)",
    "garagem (unidade autônoma) de prédio de garagens":"garagem, em prédio de garagens",
    "garagem (unidade autônoma) em edifício em condomínio de escritórios, consultórios ou misto":"garagem em edifício de escritórios, consultórios ou misto",
    "garagem (unidade autônoma) em edifício em condomínio de uso exclusivamente residencial":"garagem em edifício de uso exclusivamente residencial",
    "outras edificações de uso coletivo, com utilização múltipla":"outras edificações do tipo (uso coletivo), com utilização múltipla",
    "outras edificações de uso comercial, com utilização múltipla":"outras edificações do tipo (uso comércio), com utilização múltipla",
    "outras edificações de uso de serviço, com utilização múltipla":"outras edificações do tipo (uso serviço), com utilização múltipla",
    "outras edificações de uso especial, com utilização múltipla":"outras edificações do tipo (uso especial), com utilização múltipla",
    "prédio com uso exclusivamente residencial, não em condomínio":"prédio de apartamento, não em condomínio, de uso exclusivamente residencial",
    "prédio com uso misto, predominância de uso não residencial, não em condomínio":"prédio de escritório, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios) com ou sem loja (predominância comercial)",
    "prédio com uso misto, predominância de uso residencial, não em condomínio":"prédio de apartamento, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios), com ou sem loja (predominância residencial)",
    "prédio de escritório ou consultório, com ou sem lojas, não em condomínio":"prédio de escritório ou consultório, não em condomínio, com ou sem lojas",
    "flat de uso comercial (semelhante a hotel)":"flat, não residencial",
    "flat residencial em condomínio":"flat, residencial"
}

In [3]:
df_iptu = pd.read_csv('IPTU/IPTU_2008.zip',\
             compression='zip',\
             encoding='iso-8859-9',\
             sep=';',\
             decimal=',',\
             error_bad_lines=False)

b'Skipping line 2440049: expected 35 fields, saw 43\n'


In [4]:
# Obter o SQ (Setor e Quadra)
df_iptu['setor'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[0:3]
df_iptu['quadra'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[3:6]

# Calculando chave 'SQ'
df_iptu['sq'] = \
df_iptu['setor'] + df_iptu['quadra']

# Normalizando o lote condominial
df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] != '00', 'lote'] = '0000'
df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'lote'] = \
df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'NUMERO DO CONTRIBUINTE'].str[6:10]

# Individualizando a testada e pavimentos por lote (SQLC)
df_iptu['sqlc'] = df_iptu['setor'] + \
df_iptu['quadra'] + df_iptu['lote'] + \
df_iptu['NUMERO DO CONDOMINIO'].str[0:2]

df_iptu['AREA CONSTRUIDA'] = pd.to_numeric(df_iptu['AREA CONSTRUIDA'], errors ='coerce')
df_iptu['AREA OCUPADA'] = pd.to_numeric(df_iptu['AREA OCUPADA'], errors ='coerce')
df_iptu['AREA DO TERRENO'] = pd.to_numeric(df_iptu['AREA DO TERRENO'], errors ='coerce')
df_iptu['FRACAO IDEAL'] = pd.to_numeric(df_iptu['FRACAO IDEAL'], errors ='coerce')
df_iptu['QUANTIDADE DE PAVIMENTOS'] = pd.to_numeric(df_iptu['QUANTIDADE DE PAVIMENTOS'], errors ='coerce')
df_iptu['TESTADA PARA CALCULO'] = pd.to_numeric(df_iptu['TESTADA PARA CALCULO'], errors ='coerce')

# Adequando variáveis à fração ideal
df_iptu['area_terreno'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA DO TERRENO']
df_iptu['area_ocupada'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA OCUPADA']

# Adequar os usos ao padrão estabelecido
df_iptu.loc[df_iptu['TIPO DE USO DO IMOVEL']\
            .isin(usos),\
            'TIPO DE USO DO IMOVEL'] = \
df_iptu[df_iptu['TIPO DE USO DO IMOVEL']\
             .isin(usos)]['TIPO DE USO DO IMOVEL']\
.apply(lambda x: usos[x])

## Carregando e pré-processando quadras

In [5]:
# Carregando geometria das quadras fiscais
gdf_quadras = gpd.GeoDataFrame.from_file(f'zip://GIS/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF')

In [6]:
gdf_quadras['sq'] = gdf_quadras.qd_setor + gdf_quadras.qd_fiscal

In [7]:
gdf_quadras = gdf_quadras.loc[gdf_quadras.qd_tipo == 'F'][['sq', 'geometry']].dissolve(by='sq')

In [8]:
# gdf_quadras.geometry = gdf_quadras.representative_point()

## Agregando e espacializando dado de densidade contruída

In [9]:
# Agregar por SQ e ANO DO EXERCICIO
iptu_agg = df_iptu[['sq', \
                    'AREA CONSTRUIDA',
                    'area_terreno']]\
.groupby('sq')\
.agg({'AREA CONSTRUIDA': 'sum',
     'area_terreno': 'sum'})\
.rename(columns = {'AREA CONSTRUIDA': 'area_construida'})

In [10]:
iptu_agg['densidade'] = round(iptu_agg['area_construida'] / iptu_agg['area_terreno'], 2)

In [11]:
iptu_agg['densidade']

sq
001003    1.82
001004    1.43
001005    1.78
001006    2.25
001007    1.60
          ... 
310116    1.03
310117    1.24
310118    1.41
310119    0.00
310120    0.00
Name: densidade, Length: 43440, dtype: float64

In [12]:
# iptu_agg['densidade'] = iptu_agg['densidade'].round(decimals = 2).astype('float16')

In [13]:
# Gerando GeoDataFrame
gdf_iptu_agg = gdf_quadras.merge(iptu_agg, on='sq', how='right')

In [15]:
# gdf_iptu_agg.loc['171198']

In [16]:
# df_iptu.loc[df_iptu.sq == '171198']

In [17]:
gdf_iptu_agg.drop(columns=['area_construida', 'area_terreno'])

,geometry,densidade
sq,,
001003,"POLYGON ((333415.056 7396354.502, 333412.306 7...",1.82
001004,"POLYGON ((333547.169 7396521.849, 333542.688 7...",1.43
001005,"POLYGON ((333559.635 7396520.416, 333574.733 7...",1.78
001006,"POLYGON ((333599.160 7396343.566, 333593.816 7...",2.25
001007,"POLYGON ((333689.258 7396305.230, 333688.459 7...",1.60
...,...,...
310116,"MULTIPOLYGON (((332734.442 7383503.772, 332721...",1.03
310117,"MULTIPOLYGON (((333231.482 7383452.112, 333232...",1.24
310118,"POLYGON ((332710.926 7384374.222, 332722.074 7...",1.41


In [18]:
del df_iptu

In [19]:
del gdf_iptu_agg

In [20]:
# gdf_iptu_agg.to_file('resultados/seila.gpkg', driver='GPKG')

## Gera raster densidade construtiva para cada ano

In [21]:
from pathlib import Path
import os
import numpy as np

In [22]:
densidades = []

In [23]:
for path in Path('IPTU').rglob('*.zip'):
    print(path.name)
    
    # 1. Carregar dados do arquivo
    # OBS.: O IPTU de 2016 está separado por ',' e não por ';'
    if path.name == "IPTU_2016.zip":
        df_iptu = pd.read_csv(f'{path}',\
                     compression='zip',\
                     encoding='iso-8859-9',\
                     sep=',',\
                     decimal=',',\
                     error_bad_lines=False)
    else:
        df_iptu = pd.read_csv(f'{path}',\
                     compression='zip',\
                     encoding='iso-8859-9',\
                     sep=';',\
                     decimal=',',\
                     error_bad_lines=False)
        
    # Obter o SQ (Setor e Quadra)
    df_iptu['setor'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[0:3]
    df_iptu['quadra'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[3:6]

    # Calculando chave 'SQ'
    df_iptu['sq'] = \
    df_iptu['setor'] + df_iptu['quadra']

    # Normalizando o lote condominial
    df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] != '00', 'lote'] = '0000'
    df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'lote'] = \
    df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'NUMERO DO CONTRIBUINTE'].str[6:10]

    # Individualizando a testada e pavimentos por lote (SQLC)
    df_iptu['sqlc'] = df_iptu['setor'] + \
    df_iptu['quadra'] + df_iptu['lote'] + \
    df_iptu['NUMERO DO CONDOMINIO'].str[0:2]

    df_iptu['AREA CONSTRUIDA'] = pd.to_numeric(df_iptu['AREA CONSTRUIDA'], errors ='coerce')
    df_iptu['AREA OCUPADA'] = pd.to_numeric(df_iptu['AREA OCUPADA'], errors ='coerce')
    df_iptu['AREA DO TERRENO'] = pd.to_numeric(df_iptu['AREA DO TERRENO'], errors ='coerce')
    df_iptu['FRACAO IDEAL'] = pd.to_numeric(df_iptu['FRACAO IDEAL'], errors ='coerce')
    df_iptu['QUANTIDADE DE PAVIMENTOS'] = pd.to_numeric(df_iptu['QUANTIDADE DE PAVIMENTOS'], errors ='coerce')
    df_iptu['TESTADA PARA CALCULO'] = pd.to_numeric(df_iptu['TESTADA PARA CALCULO'], errors ='coerce')

    # Adequando variáveis à fração ideal
    df_iptu['area_terreno'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA DO TERRENO']
    df_iptu['area_ocupada'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA OCUPADA']

    # Adequar os usos ao padrão estabelecido
    df_iptu.loc[df_iptu['TIPO DE USO DO IMOVEL']\
                .isin(usos),\
                'TIPO DE USO DO IMOVEL'] = \
    df_iptu[df_iptu['TIPO DE USO DO IMOVEL']\
                 .isin(usos)]['TIPO DE USO DO IMOVEL']\
    .apply(lambda x: usos[x])

    # Agregar por SQ e ANO DO EXERCICIO
    iptu_agg = df_iptu[['sq', \
                        'AREA CONSTRUIDA',
                        'area_terreno',
                        'area_ocupada',
                        'QUANTIDADE DE PAVIMENTOS',
                        'TESTADA PARA CALCULO']]\
    
    # Agregar por SQ e ANO DO EXERCICIO
    iptu_agg = df_iptu[['sq', \
                        'AREA CONSTRUIDA',
                        'area_terreno']]\
    .groupby('sq')\
    .agg({'AREA CONSTRUIDA': 'sum',
         'area_terreno': 'sum'})\
    .rename(columns = {'AREA CONSTRUIDA': 'area_construida'})
    
    # Calculando a densidade construtiva
    iptu_agg[f'{path.name[-8:-4]}'] = round(iptu_agg['area_construida'] \
                                      / iptu_agg['area_terreno'], 2)
#     iptu_agg[f'{path.name[-8:-4]}'] = iptu_agg[f'{path.name[-8:-4]}']\
#                                       .round(decimals = 2).astype('float16')
    
    # Removendo colunas sobressalentes
    densidades.append(iptu_agg.drop(columns=['area_construida', 'area_terreno']))
    
    # Gerando GeoDataFrame
    #gdf_iptu_agg = gdf_quadras.merge(iptu_agg, on='sq', how='right')

IPTU_2010.zip


b'Skipping line 2445507: expected 35 fields, saw 43\n'
/home/fernando/miniconda3/envs/geoplot/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


IPTU_1996.zip


/home/fernando/miniconda3/envs/geoplot/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


IPTU_2007.zip


...

b'Skipping line 2440049: expected 35 fields, saw 43\n'


IPTU_2003.zip


b'Skipping line 2288886: expected 35 fields, saw 43\n'


IPTU_2012.zip


b'Skipping line 2526350: expected 35 fields, saw 43\n'


In [64]:
df_densidades = \
pd.concat(densidades, axis=1).reindex(columns=sorted(pd.concat(densidades, axis=1).columns))

In [65]:
# df_densidades.loc['169197']

In [66]:
df_densidades.replace([np.inf, -np.inf], np.nan, inplace=True)

In [67]:
df_densidades = df_densidades.interpolate(method='linear', limit_direction='forward', axis=1)

In [68]:
df_densidades.loc['169197']

1995     NaN
1996     NaN
1997     NaN
1998     NaN
1999     NaN
2000     NaN
2001     NaN
2002     NaN
2003     NaN
2004     NaN
2005     NaN
2006     NaN
2007     NaN
2008     NaN
2009     NaN
2010     NaN
2011     NaN
2012     NaN
2013     NaN
2014     NaN
2015     NaN
2016    3.46
2017    4.22
2018    4.22
2019    4.22
2020    4.22
Name: 169197, dtype: float64

In [69]:
# df_densidades.loc['169197']

In [70]:
# df_densidades.loc['171198']

In [71]:
# Gerando GeoDataFrame
gdf_iptu_agg = gdf_quadras.merge(df_densidades, on='sq', how='left')

In [72]:
# gdf_iptu_agg.loc['169197']

In [73]:
gdf_iptu_agg.to_file('resultados/densidade_construtiva/IPTU_quadras_densidade_construtiva.gpkg', driver='GPKG')

In [31]:
# np.arange(1995, 2021)

In [32]:
# gdf_iptu_agg.replace([np.inf, -np.inf], np.nan, inplace=True)

In [33]:
# gdf_iptu_agg.iloc[45142].drop('geometry').std()

In [ ]:
# gdf_iptu_agg.iloc[45142].drop('geometry')

In [ ]:
# gdf_iptu_agg.iloc[45142].drop('geometry').ewm(com=0.3).mean()